In [1]:
%%capture
!pip install nltk transformers
!pip install sentencepiece

In [2]:
import nltk
from nltk.corpus import brown
from transformers import TFDistilBertModel, DistilBertTokenizer, DistilBertConfig
import numpy as np
import csv
from nltk.tokenize import word_tokenize
import os
from tqdm import tqdm
import pandas as pd
import torch

import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from tensorflow.keras.mixed_precision import Policy, set_global_policy

# Set global policy
policy = Policy('mixed_float16')
set_global_policy(policy)

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [29]:
train_df = pd.read_csv("/kaggle/input/preprocessed-reuters/preprocessed_training_data.csv")
test_df = pd.read_csv("/kaggle/input/preprocessed-reuters/preprocessed_testing_data.csv")

In [35]:
top6_cat = ['earn', 'acq', 'interest', 'crude', 'trade', 'money-fx']
train_df = train_df[train_df['Category'].isin(top6_cat)]
test_df = test_df[test_df['Category'].isin(top6_cat)]

In [7]:
nltk.download('brown')
nltk.download('punkt')

[nltk_data] Downloading package brown to /usr/share/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
model_name = "distilbert-base-uncased"

original_config = DistilBertConfig.from_pretrained(model_name)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [9]:
# Create a new configuration with reduced hidden size
reduced_hidden_size = 64  # Replace with your desired hidden size
reduced_num_hidden_layers = 2  # Choose a smaller number
reduced_num_attention_heads = 2  # Choose a smaller number
new_config = DistilBertConfig(
    vocab_size=original_config.vocab_size,
    hidden_size=reduced_hidden_size,
    num_hidden_layers=reduced_num_hidden_layers,
    num_attention_heads=reduced_num_attention_heads,
)

In [10]:
#model = TFDistilBertModel.from_pretrained(model_name, config=new_config, ignore_mismatched_sizes=True)
model = TFDistilBertModel.from_pretrained(model_name)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [11]:
def embed_words(articles):
    final_embeddings_list = []
    for text in tqdm(articles):
        tokens = tokenizer.batch_encode_plus(text, padding='max_length', truncation=True, return_tensors='tf', max_length=64)
        outputs = model(tokens['input_ids'])
        embeddings = outputs.last_hidden_state
        averaged_embeddings = np.mean(embeddings, axis=0)
    #     print(len(averaged_embeddings))
        final_embeddings_list.append(averaged_embeddings.flatten())
    
    return final_embeddings_list

In [12]:
# dataset_path = r"/content/drive/MyDrive/Reuters-21578/reuters/reuters/reuters/training"
# dataset_dirs = os.listdir(dataset_path)

In [13]:
# data = []
# for i in tqdm(dataset_dirs):
#     with open(f"{dataset_path}/{i}", 'r') as f:
#         content = f.read()
#         data.append(content)

In [14]:
train_df.head()

,Text,Category
1,champion product ch approv stock split champio...,earn
2,comput termin system cpml complet sale comput ...,acq
3,cobanco inc cbco year net shr 34 ct v 119 dlr ...,earn
4,ohio mattress omt may lower 1st qtr net ohio m...,acq
5,intern inc 2nd qtr jan 31 oper shr loss two ct...,earn


In [15]:
# words = []
# for text in train_df['Text']:
#   temp_words = word_tokenize(text)
#   words.append(temp_words)

In [16]:
# from itertools import chain

# words_flattened = list(chain(*words))


In [17]:
# len(words_flattened)

In [18]:
# words_flattened = [str(element) for element in words_flattened]

In [19]:
# %tensorflow_version 2.x
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.13.0
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [20]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [21]:
with tf.device('/device:GPU:0'):
  embeddings = embed_words(train_df['Text'].to_list())

100%|██████████| 5636/5636 [51:01<00:00,  1.84it/s]  


In [22]:
len(embeddings)

5636

In [23]:
type(embeddings)

list

In [24]:
np.save('training_embeddings.npy', embeddings)


In [26]:
train_y = train_df['Category'].to_numpy()

In [27]:
np.save('training_labels.npy', train_y)

In [36]:
with tf.device('/device:GPU:0'):
  test_embeddings = embed_words(test_df['Text'].to_list())

100%|██████████| 2242/2242 [18:48<00:00,  1.99it/s]


In [37]:
np.save('testing_embeddings.npy', test_embeddings)


In [38]:
test_y = test_df['Category'].to_numpy()

In [39]:
np.save('testing_labels.npy', test_y)